In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the mock data
data = pd.read_csv('large_mock_energy_data.csv')

# Convert timestamp to datetime format
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Create a Dash application
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='Real-Time Energy Consumption Monitoring Dashboard'),

    # Dropdown for selecting the metric to display
    html.Label('Select Metric:'),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Energy Consumption', 'value': 'energy_consumption'},
            {'label': 'Temperature', 'value': 'temperature'}
        ],
        value='energy_consumption',  # Default value
        clearable=False
    ),

    # Graph to display the data
    dcc.Graph(
        id='time-series-graph'
    ),

    # Slider for selecting the date range
    dcc.RangeSlider(
        id='date-slider',
        min=0,
        max=len(data) - 1,
        value=[0, len(data) - 1],
        marks={i: data['timestamp'][i].strftime('%Y-%m-%d') for i in range(0, len(data), int(len(data)/10))},
        step=1
    )
])

# Define the callback to update the graph based on the dropdown and slider input
@app.callback(
    Output('time-series-graph', 'figure'),
    [Input('metric-dropdown', 'value'),
     Input('date-slider', 'value')]
)
def update_graph(selected_metric, date_range):
    # Filter data based on the selected date range
    filtered_data = data.iloc[date_range[0]:date_range[1]]

    # Create a time-series plot
    fig = px.line(filtered_data, x='timestamp', y=selected_metric,
                  title=f'{selected_metric} over Time',
                  labels={'timestamp': 'Timestamp', selected_metric: selected_metric.capitalize()})

    # Add additional layout settings
    fig.update_layout(transition_duration=500, xaxis_title='Timestamp', yaxis_title=selected_metric.capitalize())

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)
